In [1]:
from os import environ, chdir

from keras.preprocessing.image import ImageDataGenerator

from keras import models, layers, optimizers, callbacks

import tensorflow as tf

import h5py

from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D

from keras.models import Sequential, Model

from keras.layers.normalization import BatchNormalization

environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#chdir(r'Image-classification/train')

Using TensorFlow backend.


In [2]:
!git clone https://github.com/mskcsetce/Image-classification.git

Cloning into 'Image-classification'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 528 (delta 1), reused 0 (delta 0), pack-reused 520
Receiving objects: 100% (528/528), 387.57 MiB | 35.61 MiB/s, done.
Resolving deltas: 100% (38/38), done.
Checking out files: 100% (424/424), done.


In [3]:
# Setting Image and Data Generators
train_idg = ImageDataGenerator(rescale=1. / 255,
                               zoom_range=[1.0, 1.25],
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               fill_mode='reflect')

train_g = train_idg.flow_from_directory(directory="Image-classification/train/",
                                        target_size=(100,100),
                                        class_mode='binary',
                                        batch_size=108,
                                        shuffle=True)

valid_idg = ImageDataGenerator(rescale=1. / 255)

valid_g = valid_idg.flow_from_directory(directory='Image-classification/valid/',
                                        target_size=(100,100),
                                        class_mode='binary',
                                        batch_size=50,
                                        shuffle=True)

Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [4]:
# In[5]:

# CNN Architecture
my_model = models.Sequential()
my_model.add(layers.Conv2D(filters=8, kernel_size=(3,3), strides=(1, 1),
                           input_shape=(100, 100, 3)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Dropout(rate=0.4))

my_model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Dropout(rate=0.4))

my_model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
my_model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Flatten())

my_model.add(layers.Dropout(rate=0.4))

my_model.add(layers.Dense(units=100))
my_model.add(layers.Activation('relu'))

my_model.add(layers.Dense(units=1))
my_model.add(layers.Activation('sigmoid'))
print(my_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 8)         224       
_________________________________________________________________
activation_1 (Activation)    (None, 98, 98, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 8)         584       
_________________________________________________________________
activation_2 (Activation)    (None, 96, 96, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 46, 16)        1168      
__________

In [0]:
# Model Loss function and Optimizer method
compile = my_model.compile(optimizer=optimizers.Adadelta(lr=0.15), loss='binary_crossentropy',
                           metrics=['accuracy'])


In [0]:
# Settting Callbacks
#chdir(r'D:\hackathon')
check_p = callbacks.ModelCheckpoint(filepath='hydrangea_cnn_{val_acc:.2f}.h5',
                                    monitor='val_acc', verbose=1,
                                    save_best_only=True, save_weights_only=False)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=3,
                                        verbose=1, cooldown=2)
callb_l = [check_p, reduce_lr]

In [7]:
# Training Options
fit = my_model.fit_generator(generator=train_g, steps_per_epoch=2, epochs=2, verbose=1,
                             callbacks=callb_l, validation_data=valid_g, validation_steps=4)

Epoch 1/2
2/2 [==============================] - 15s 7s/step - loss: 0.6935 - acc: 0.4873 - val_loss: 0.6919 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to hydrangea_cnn_0.50.h5
Epoch 2/2
2/2 [==============================] - 13s 6s/step - loss: 0.6939 - acc: 0.5021 - val_loss: 0.6919 - val_acc: 0.5000

Epoch 00002: val_acc did not improve from 0.50000


In [0]:
# Saving Model
my_model.save(filepath='Image-classification/models/hydrangea_cnn.h5', overwrite=True)


In [15]:

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs= range(len(acc))

plt.plot(epochs, acc, 'b', label='training acc')
plt.plot(epochs, val_acc, 'r', label='validation loss')
plt.title('training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='training loss')
plt.plot(epochs, val_loss, 'r', label='training loss')
plt.title('training and validation loss')
plt.legend()

plt.show()



NameError: ignored